In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
%run -i /home/chuwen/cora/facility-loc-inventory/test_cp_gfc.py

2024-04-23 14:51:10,564	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[facinv] 2024-04-23 14:51:10,657: -- The FACINV python package --
[facinv] 2024-04-23 14:51:10,658:   LLLGZ, 2023 (c)   
[facinv] 2024-04-23 14:51:10,658: :solution      to ./out
[facinv] 2024-04-23 14:51:10,659: :data          to ./data
[facinv] 2024-04-23 14:51:10,659: :logs and tmps to ./tmp
{
  "T": 1,
  "add_cardinality": 0,
  "add_distance": 0,
  "add_in_upper": 1,
  "backend": "gurobi",
  "backorder": 0,
  "backorder_sku_unit_cost": 5000,
  "bool_use_ncg": 1,
  "capacity": 1,
  "capacity_node_ratio": 1,
  "capacity_ratio": 1,
  "cardinality": 0,
  "cardinality_limit": 30,
  "cg_itermax": 10,
  "cg_method_mip_heuristic": -1,
  "cg_mip_recover": 1,
  "cg_rmp_mip_iter": 10,
  "check_cost_cg": 0,
  "conf_label": 3,
  "covering": 1,
  "cus_num": 4,
  "d": 150,
  "del_col_freq": 3,
  "del_col_stra": 1,
  "demand_type": 1,
  "distance": 0,
  "distance_limit": 5000,
  "edge_lb": 1,
  "fixed_cost": 0,
  "fpath": "data/data_0401_0inv.xlsx",
  "holding_sku_unit_cost": 1,
  "if_del_col": nu

FileNotFoundError: [Errno 2] No such file or directory: 'facility-loc-inventory/data/small_instance_generate_2/node_info/facility.csv'

## relax intermediate nodes

We now relax all intermediate node binary variables,
- this should be better than pure LP relaxation

This gives a `partial LP relaxation`


In [3]:
for k, v in model.variables['select_edge'].items():
    if k[1].end.type != "C":
        v.setType(COPT.CONTINUOUS)
        print(f"relaxing {k}")

model.solve()

lpval = model.get_model_objval()
os.makedirs(f"sol_lp_{pick_instance}/", exist_ok=True)

relaxing (0, Edge_P0001-GZA1_T0015_(P0001-GZA1, T0015))
relaxing (0, Edge_P0001-GZA2_T0015_(P0001-GZA2, T0015))
relaxing (0, Edge_T0001_T0002_(T0001, T0002))
relaxing (0, Edge_T0001_T0003_(T0001, T0003))
relaxing (0, Edge_T0001_T0014_(T0001, T0014))
relaxing (0, Edge_T0001_T0015_(T0001, T0015))
relaxing (0, Edge_T0001_T0019_(T0001, T0019))
relaxing (0, Edge_T0001_T0020_(T0001, T0020))
relaxing (0, Edge_T0001_T0021_(T0001, T0021))
relaxing (0, Edge_T0001_T0022_(T0001, T0022))
relaxing (0, Edge_T0001_T0023_(T0001, T0023))
relaxing (0, Edge_T0001_T0024_(T0001, T0024))
relaxing (0, Edge_T0001_T0025_(T0001, T0025))
relaxing (0, Edge_T0001_T0026_(T0001, T0026))
relaxing (0, Edge_T0002_T0019_(T0002, T0019))
relaxing (0, Edge_T0002_T0020_(T0002, T0020))
relaxing (0, Edge_T0002_T0021_(T0002, T0021))
relaxing (0, Edge_T0002_T0022_(T0002, T0022))
relaxing (0, Edge_T0002_T0023_(T0002, T0023))
relaxing (0, Edge_T0003_T0001_(T0003, T0001))
relaxing (0, Edge_T0003_T0002_(T0003, T0002))
relaxing (0, E

Then we see that

- CG and LP are different
- but they are worse than MIP (as expected)

In [4]:

print(
    f"""
--- summary ---------------------------
lp relaxation: {lpval},
mip          : {mipval},
cg           : {np_cg.RMP_model.objval}
"""
)



--- summary ---------------------------
lp relaxation: 7335.683553451619,
mip          : 7373.645201525189,
cg           : 7372.590159159727



## GFC in `partial LP relaxation`
Our goal is to cutoff these solutions

In [5]:
from tqdm.notebook import tqdm as tqdm_notebook

import utils
from cp_gfc import *

- Let `inbound edges` as $N_2$, `out` as $N_1$,
- net inventory as $d$,
we see, 

\begin{align*}
\begin{aligned}
& \sum_{j \in N_1} x_j-\sum_{j \in N_2} x_j \leq d, \\
& l_j y_j \leq x_j \leq u_j y_j, \quad y_j \in\{0,1\}, \quad j \in N_1 \cup N_2
\end{aligned}
\end{align*}

where each $x_j$ is the total flow summarizing all SKUs

In [6]:
model.model.setParam("Logging", 0)
# get the first solutions
df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
lpval = model.model.objval

[facinv] 2023-10-29 22:29:06,219: table warehouse_total_avg_inventory failed
[facinv] 2023-10-29 22:29:06,230: saving finished


In [11]:
k = 0
while k < 10:
    for _, row in query_fractional_inter_edges(df_edges).iterrows():
        start, end, t = row["obj_start"], row["obj_end"], row["t"]
        break

    N1 = [e for e in utils.get_out_edges(network, end) if e.capacity < 1e6]
    N2 = [e for e in utils.get_in_edges(network, end) if e.capacity < 1e6]

    x = model.variables["sku_flow"]
    y = model.variables["select_edge"]
    d = 0
    cutv, lbdv, subsets = seperation_gcf(model, x, y, t, N1, N2, d, dump=False, verbose=False)
    if cutv > 1e-2:
        expr, cut_value, lbd, bool_voilate = eval_cut_c3(x, y, t, d, *subsets, lbdv=lbdv)
        model.model.addConstr(expr <= 0)
        model.model.solve()
        df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
    else:
        print("cannot find")
        break
    
    print(f"""
--- {k} ---
{start, end, t}
{model.model.objval - lpval}
    """)
    lpval = model.model.objval
    k += 1

Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.01
0 62.81875505785836 -62.81875505785836
this is unmatched with separation model
[facinv] 2023-10-29 22:36:15,388: table warehouse_total_avg_inventory failed
[facinv] 2023-10-29 22:36:15,399: saving finished

--- 0 ---
(T0015, T0014, 0)
0.0
    
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.01
0 62.81875505785836 -62.81875505785836
this is unmatched with separation model
[facinv] 2023-10-29 22:36:16,320: table warehouse_total_avg_inventory failed
[facinv] 2023-10-29 22:36:16,332: saving finished

--- 0 ---
(T0015, T0014, 0)
0.0
    
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.01
0 62.81875505785836 -62.81875505785836
this is unmatched with separation model
[facinv] 2023-10-29 22:36:17,217: table warehouse_total_avg_inventory failed
[facinv] 2023-10-29 22:36:17,229: saving finished

--- 0 ---
(T0015, T0014, 0)
0.0
    
Set parameter NonConvex to value 2
Set parameter MIPGap to

AttributeError: Unable to retrieve attribute 'x'

In [ ]:
df_edges

In [ ]:
subsets